# Load data

In [1]:
from datetime import datetime
from collections import defaultdict

from vnpy_evo.trader.database import get_database, DB_TZ
from vnpy_evo.trader.constant import Interval
from vnpy_evo.trader.object import BarData
from vnpy_evo.trader.utility import extract_vt_symbol

db = get_database()

In [2]:
# Load data
vt_symbols = ["BTCUSDT.BINANCE", "ETHUSDT.BINANCE"]

history: dict[datetime, dict[str, BarData]] = defaultdict(dict)

for vt_symbol in vt_symbols:
    symbol, exchange = extract_vt_symbol(vt_symbol)

    bars: list[BarData] = db.load_bar_data(
        symbol=symbol,
        exchange=exchange,
        interval=Interval.MINUTE,
        start=datetime(2024, 8, 1, tzinfo=DB_TZ),
        end=datetime(2024, 9, 10, tzinfo=DB_TZ)
    )

    for bar in bars:
        history[bar.datetime][vt_symbol] = bar
            

# Initialize table

In [3]:
# Choose which table to test
from vnpy_novastrategy import (
    # LiveDataTable as DataTable,           
    BacktestingDataTable as DataTable
)

# Create table
table = DataTable(
    vt_symbols=vt_symbols,
    size=200,
    interval=Interval.MINUTE,
    extra_fields=["active_volume", "active_turnover", "trade_count"]
)


In [ ]:
# Add simple feature
table.add_feature("range", "high_price - low_price")

In [ ]:
# Add feature with parameters
for fast_window, slow_window in [
    (5, 25),
    (10, 50),
    (20, 100)
]:
    table.add_feature(
        name=f"ma_gap_{fast_window}_{slow_window}",
        expression=f"(ts_mean(close_price, {fast_window}) / ts_mean(close_price, {slow_window}) - 1) * 100"
    )

In [ ]:
# Add feature with complex expression
mfi_period = 15
neutral_period = 20

_mfi = f"ta_mfi(high_price, low_price, close_price, volume, {mfi_period})"
_mfi_mean = f"ts_mean({_mfi}, {neutral_period})"
_mfi_std = f"ts_std({_mfi}, {neutral_period})"
_mfi_zscore = f"({_mfi} - {_mfi_mean}) / {_mfi_std}"
expression = f"rank({_mfi_zscore})"
print(expression)

table.add_feature("ranked_mfi", expression)

In [ ]:
# Add feature with complex expressions
mfi_period = 15
neutral_period = 20

_mfi = f"ta_mfi(high_price, low_price, close_price, volume, {mfi_period})"
_mfi_mean = f"ts_mean({_mfi}, {neutral_period})"
_mfi_std = f"ts_std({_mfi}, {neutral_period})"
_mfi_zscore = f"({_mfi} - {_mfi_mean}) / {_mfi_std}"
expression = f"rank({_mfi_zscore})"
print(expression)

table.add_feature("ranked_mfi", expression)

In [ ]:
# Add feature with intermediate variables
short_window = 5
long_window = 20
signal_window = 50

table.add_feature("short_ma", f"ts_mean(close_price, {short_window})")
table.add_feature("long_ma", f"ts_mean(close_price, {long_window})")
table.add_feature("buy_signal_mask", f"ts_greater_than(short_ma, long_ma) * ts_greater_than(close_price, short_ma)")
table.add_feature("buy_proportion", f"ts_sum(buy_signal_mask, {signal_window}) / {signal_window}")
table.add_feature("buy_signal_proportion_rank", "rank(buy_proportion)")

In [ ]:
# Add feature with parameters
for rsi_window in [15, 20, 25]:
    for rsi_threshold in range(10, 80, 10):
        name = f"rsi_above_threshold_{rsi_window}_{rsi_threshold}"

        _rsi = f"ta_rsi(close_price, {rsi_window})"
        _rsi_above_threshold = f"ts_greater_than({_rsi}, {rsi_threshold})"
        expression = f"cs_sum({_rsi_above_threshold}) / cs_count(close_price)"

        table.add_feature(name, expression)


In [24]:
# Add feature with cross-sectional zscore
table.add_feature("ma20_ma60", "cs_zscore(ts_mean(close_price, 20) / ts_mean(close_price, 60))")
table.add_feature("ma20_ma120", "cs_zscore(ts_mean(close_price, 20) / ts_mean(close_price, 120))")
table.add_feature("rsi_14", "cs_zscore(ta_rsi(close_price, 14))")

table.add_feature("label", "cs_zscore(ts_delay(close_price, -20) / close_price - 1)")

# For live trading usage

In [ ]:
# Update bars into table    
for dt, bars in history.items():
    table.update_bars(bars)

In [ ]:
# Get latest dataframe
df = table.get_df()
df.tail(20)

# For backtesting usage

In [5]:
# Update entire history into table
table.update_history(list(history.values()))

In [6]:
# Update bars into table    
for dt, bars in history.items():
    table.update_bars(bars)

In [23]:
# Get latest dataframe
df = table.get_df()
df.tail(20)

open_price  high_price  low_price  \
datetime                  vt_symbol                                            
2024-09-09 23:51:00+08:00 BTCUSDT.BINANCE    55414.20    55442.00   55386.10   
                          ETHUSDT.BINANCE     2294.74     2296.04    2293.36   
2024-09-09 23:52:00+08:00 BTCUSDT.BINANCE    55403.20    55403.20   55337.40   
                          ETHUSDT.BINANCE     2293.70     2293.80    2291.18   
2024-09-09 23:53:00+08:00 BTCUSDT.BINANCE    55338.40    55350.00   55315.30   
                          ETHUSDT.BINANCE     2291.27     2291.63    2290.60   
2024-09-09 23:54:00+08:00 BTCUSDT.BINANCE    55340.00    55376.50   55340.00   
                          ETHUSDT.BINANCE     2291.07     2292.48    2291.07   
2024-09-09 23:55:00+08:00 BTCUSDT.BINANCE    55360.80    55389.30   55352.70   
                          ETHUSDT.BINANCE     2292.08     2293.16    2291.69   
2024-09-09 23:56:00+08:00 BTCUSDT.BINANCE    55353.60    55412.50   55351.20   
                          ETHUSDT.BINANCE     2291.69     2293.87    2291.50   
2024-09-09 23:57:00+08:00 BTCUSDT.BINANCE    55412.50    55412.50   55373.30   
                          ETHUSDT.BINANCE     2293.71     2293.90    2292.46   
2024-09-09 23:58:00+08:00 BTCUSDT.BINANCE    55396.10    55396.10   55358.40   
                          ETHUSDT.BINANCE     2293.89     2293.89    2292.00   
2024-09-09 23:59:00+08:00 BTCUSDT.BINANCE    55362.00    55362.00   55340.00   
                          ETHUSDT.BINANCE     2292.01     2292.01    2291.07   
2024-09-10 00:00:00+08:00 BTCUSDT.BINANCE    55360.30    55372.70   55319.40   
                          ETHUSDT.BINANCE     2291.61     2292.33    2290.00   

                                           close_price    volume  \
datetime                  vt_symbol                                
2024-09-09 23:51:00+08:00 BTCUSDT.BINANCE     55403.10   209.694   
                          ETHUSDT.BINANCE      2293.70  2451.794   
2024-09-09 23:52:00+08:00 BTCUSDT.BINANCE     55338.30   143.372   
                          ETHUSDT.BINANCE      2291.27  1050.940   
2024-09-09 23:53:00+08:00 BTCUSDT.BINANCE     55340.00   131.051   
                          ETHUSDT.BINANCE      2291.06   732.711   
2024-09-09 23:54:00+08:00 BTCUSDT.BINANCE     55360.80    71.259   
                          ETHUSDT.BINANCE      2292.08   638.396   
2024-09-09 23:55:00+08:00 BTCUSDT.BINANCE     55353.70    91.905   
                          ETHUSDT.BINANCE      2291.69   952.125   
2024-09-09 23:56:00+08:00 BTCUSDT.BINANCE     55412.40   120.399   
                          ETHUSDT.BINANCE      2293.72   819.700   
2024-09-09 23:57:00+08:00 BTCUSDT.BINANCE     55396.10    90.329   
                          ETHUSDT.BINANCE      2293.90   576.716   
2024-09-09 23:58:00+08:00 BTCUSDT.BINANCE     55362.00    48.297   
                          ETHUSDT.BINANCE      2292.00   544.330   
2024-09-09 23:59:00+08:00 BTCUSDT.BINANCE     55360.30   120.094   
                          ETHUSDT.BINANCE      2291.59   819.934   
2024-09-10 00:00:00+08:00 BTCUSDT.BINANCE     55369.10   138.978   
                          ETHUSDT.BINANCE      2291.78  1457.763   

                                               turnover  open_interest  \
datetime                  vt_symbol                                      
2024-09-09 23:51:00+08:00 BTCUSDT.BINANCE  1.162116e+07              0   
                          ETHUSDT.BINANCE  5.626072e+06              0   
2024-09-09 23:52:00+08:00 BTCUSDT.BINANCE  7.936740e+06              0   
                          ETHUSDT.BINANCE  2.408868e+06              0   
2024-09-09 23:53:00+08:00 BTCUSDT.BINANCE  7.251446e+06              0   
                          ETHUSDT.BINANCE  1.678702e+06              0   
2024-09-09 23:54:00+08:00 BTCUSDT.BINANCE  3.944729e+06              0   
                          ETHUSDT.BINANCE  1.463040e+06              0   
2024-09-09 23:55:00+08:00 BTCUSDT.BINANCE  5.